# Segments

This notebook creates the trip segments outlined in Section 3 of the paper from GTFS files. 

Outline:
* Takes static GTFS schedules and generates trip segments.
* Maps Inrix, HERE and OSM to the segments.

In [ ]:
import gtfs_functions as gtfs
import geopandas as gpd
import pandas as pd
import os
import zipfile
import shutil
import datetime as dt
import time
from copy import deepcopy
import datetime
from shapely.geometry import Point, LineString, Polygon, asShape, mapping
import requests
from plotly import graph_objs as go
import numpy as np
from shapely.ops import cascaded_union
import folium
import math
import requests
import concurrent.futures
import json
import pymongo

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

os.chdir("../")
print(f"Current working directory: {os.getcwd()}")

In [ ]:
# global parameters
GTFS_FILES = ['2019-08-18', '2020-04-13']

In [ ]:
# hyperparameters

BUF = 20
MAX_TRAJ_DIFF = 5
MIN_NUM = 3
INTERPOLATE_DIST = 10

In [ ]:
# input file paths

GTFS_UNZIPPED_DIR = os.path.join(os.getcwd(), 'data', 'gtfs', 'schedules_unzipped')
GTFS_ZIPPED_DIR = os.path.join(os.getcwd(), 'data', 'gtfs', 'schedules_zipped')
INRIX_SHAPES_PATH = os.path.join(os.getcwd(), 'data', 'inrix', 'USA_Tennessee.geojson')

In [ ]:
# output file path

SEGMENTS_PATH = os.path.join(os.getcwd(), 'output_r', 'segments', 'segments.pkl')
TEMP_PATH = os.path.join(os.getcwd(), 'output_r', 'temp', 'segments.pkl')

# 1. Generate the Segments

In [ ]:
def load_gtfs(gtfs_filename, include_calendar=False):
    dir_path = os.path.join(GTFS_UNZIPPED_DIR, gtfs_filename)
    output_path = os.path.join(GTFS_ZIPPED_DIR, gtfs_filename)
    shutil.make_archive(output_path, 'zip', dir_path)
    
    file_path = os.path.join(GTFS_ZIPPED_DIR, f"{gtfs_filename}.zip")
    routes, stops, stop_times, trips, shapes = gtfs.import_gtfs(file_path, busiest_date=False)
    routes = routes.drop_duplicates()
    stops = stops.drop_duplicates()
    stop_times = stop_times.drop_duplicates()
    trips = trips.drop_duplicates()
    shapes = shapes.drop_duplicates()
    if include_calendar:
        file_path = os.path.join(GTFS_UNZIPPED_DIR, gtfs_filename, 'calendar.txt')
        calendar = pd.read_csv(file_path)
        calendar['start_date'] = calendar['start_date'].apply(lambda x: dt.datetime.strptime(str(x), '%Y%m%d').date().isoformat())
        calendar['end_date'] = calendar['end_date'].apply(lambda x: dt.datetime.strptime(str(x), '%Y%m%d').date().isoformat())
        calendar = calendar[['service_id', 'start_date', 'end_date']]
        return routes, stops, stop_times, trips, shapes, calendar
    else:
        return routes, stops, stop_times, trips, shapes


def segments_per_trip(trip_id, stop_times):
    result = {'trip_id': [], 
              'segment_seq': [],
              'start_stop_id': [], 
              'start_stop_geometry': [], 
              'end_stop_id': [], 
              'end_stop_geometry': [], 
              'shape_id': [], 
              'direction_id': [], 
              'route_id': [], 
              'distance_btw_stops': []}
    stop_times_trip = stop_times[stop_times['trip_id']==trip_id].sort_values(by=['stop_sequence']).reset_index()
    for i in range(len(stop_times_trip)-1):
        start_stop = stop_times_trip.iloc[i]
        end_stop = stop_times_trip.iloc[i+1]
        
        result['trip_id'].append(trip_id)
        result['segment_seq'].append(i)
        result['start_stop_id'].append(start_stop['stop_id'])
        result['start_stop_geometry'].append(start_stop['geometry'])
        result['end_stop_id'].append(end_stop['stop_id'])
        result['end_stop_geometry'].append(end_stop['geometry'])
        result['shape_id'].append(start_stop['shape_id'])
        result['direction_id'].append(start_stop['direction_id'])
        result['route_id'].append(start_stop['route_id'])
        distance_btw_stops = end_stop['shape_dist_traveled'] - start_stop['shape_dist_traveled']
        result['distance_btw_stops'].append(distance_btw_stops)
    return pd.DataFrame(result)
        

def generate_trip_segments(gtfs_filename):
    routes, stops, stop_times, trips, shapes, calendar = load_gtfs(gtfs_filename, include_calendar=True)
    segments_gdf = gtfs.cut_gtfs(stop_times, stops, shapes)
    segments_gdf = segments_gdf.drop_duplicates(subset=['route_id', 'direction_id', 'shape_id', 'start_stop_id', 'end_stop_id'])
    
    df_list = []
    for trip_id in stop_times['trip_id'].unique():
        df = segments_per_trip(trip_id, stop_times)
        df_list.append(df)
    df = pd.concat(df_list, ignore_index=True)
    trip_segments = pd.merge(df, segments_gdf, on=['route_id', 'direction_id', 'shape_id', 'start_stop_id', 'end_stop_id'], suffixes=('','_segments_gdf'), how='left', validate='many_to_one')
    temp = trip_segments['trip_id'].apply(lambda x: get_service_range(x, trips, calendar))
    trip_segments['gtfs_start_date'] = [x[0] for x in temp]
    trip_segments['gtfs_end_date'] = [x[1] for x in temp]
    trip_segments['gtfs_filename'] = gtfs_filename
    return trip_segments

def get_service_range(trip_id, trips, calendar):
    service_id = int(trips[trips['trip_id']==trip_id].iloc[0]['service_id'])
    temp = calendar[calendar['service_id']==service_id].iloc[0]
    return temp['start_date'], temp['end_date']


def get_point(row, pref):
    return Point(row[f"{pref}_stop_lon"], row[f"{pref}_stop_lat"])


def format_segments(df, add_stop_points=True):
    df = df.set_geometry('geometry')
    df['trip_id'] = df['trip_id'].astype(int)
    df['segment_seq'] = df['segment_seq'].astype(int)
    df['start_stop_id'] = df['start_stop_id'].astype(int)
    df['end_stop_id'] = df['end_stop_id'].astype(int)
    df['direction_id'] = df['direction_id'].astype(int)
    df['route_id'] = df['route_id'].astype(str)
    df['distance_btw_stops'] = df['distance_btw_stops'].astype(float)
    df['start_stop_name'] = df['start_stop_name'].astype(str)
    df['end_stop_name'] = df['end_stop_name'].astype(str)
    df['distance_m'] = df['distance_m'].astype(float)
    df['gtfs_start_date'] = df['gtfs_start_date'].apply(lambda x: datetime.date.fromisoformat(x))
    df['gtfs_end_date'] = df['gtfs_end_date'].apply(lambda x: datetime.date.fromisoformat(x))
    df['segment_id'] = df['segment_id'].astype(str)

    if add_stop_points:
        df['start_stop_geometry'] = df.apply(lambda row: get_point(row, 'start'), axis=1)
        df['end_stop_geometry'] = df.apply(lambda row: get_point(row, 'end'), axis=1)
        df = df.drop(columns=['start_stop_lon', 'start_stop_lat', 'end_stop_lat', 'end_stop_lon'])
    else:
        df['start_stop_lon'] = df['start_stop_lon'].astype(float)
        df['end_stop_lon'] = df['end_stop_lon'].astype(float)
        df['start_stop_lat'] = df['start_stop_lat'].astype(float)
        df['end_stop_lat'] = df['end_stop_lat'].astype(float)
    return df


def merge_segments(df1, df2, gtfs_end_date):
    df_1 = deepcopy(df1)
    df_2 = deepcopy(df2)
    trip_ids = df_2['trip_id'].unique()
    for trip_id in trip_ids:
        temp1 = df_1[df_1['trip_id']==trip_id].drop(columns=['gtfs_start_date', 'gtfs_end_date']).sort_values(by=['segment_seq']).reset_index(drop=True)
        temp2 = df_2[df_2['trip_id']==trip_id].drop(columns=['gtfs_start_date', 'gtfs_end_date']).sort_values(by=['segment_seq']).reset_index(drop=True)
        if (len(temp1) > 0) & (temp1.equals(temp2)):
            df_1.loc[(df_1['trip_id']==trip_id), 'gtfs_end_date'] = gtfs_end_date
        elif (len(temp1) == 0):
            temp = df_2[df_2['trip_id']==trip_id]
            df_1 = pd.concat([df_1, temp], ignore_index=True)
        else:
            print(f"trip_id: {trip_id} is in df1 but is different when gtfs_end_date: {gtfs_end_date}")
    return df_1

In [ ]:
# Get all segments from the GTFS files

trip_segments_list = []
for gtfs_name in GTFS_FILES:
    start_time = time.time()
    print(f"Starting gtfs: {gtfs_name}")
    temp = generate_trip_segments(gtfs_name)
    trip_segments_list.append(temp)
    end_time = time.time() - start_time
    print(f"Done with gtfs: {gtfs_name}, run took {end_time} seconds")

trip_segments = pd.concat(trip_segments_list, ignore_index=True)

trip_segments['start_stop_lon'] = trip_segments['start_stop_geometry'].apply(lambda x: x.x)
trip_segments['start_stop_lat'] = trip_segments['start_stop_geometry'].apply(lambda x: x.y)
trip_segments['end_stop_lon'] = trip_segments['end_stop_geometry'].apply(lambda x: x.x)
trip_segments['end_stop_lat'] = trip_segments['end_stop_geometry'].apply(lambda x: x.y)
trip_segments = trip_segments.drop(columns=['start_stop_geometry', 'end_stop_geometry', 'stop_sequence', 'gtfs_filename'])

t = gpd.GeoDataFrame(trip_segments, geometry='geometry')

In [ ]:
# merge segments that appear in multiple GTFS files

df = format_segments(t, add_stop_points=False)
gtfs_groups = df.groupby(['gtfs_start_date','gtfs_end_date']).size().reset_index().rename(columns={0:'count'}).sort_values(by=['gtfs_start_date'])
df1 = df[(df['gtfs_start_date']==gtfs_groups.iloc[0]['gtfs_start_date']) & (df['gtfs_end_date']==gtfs_groups.iloc[0]['gtfs_end_date'])]
for i in range(1, len(gtfs_groups)):
    df2 = df[(df['gtfs_start_date']==gtfs_groups.iloc[i]['gtfs_start_date']) & (df['gtfs_end_date']==gtfs_groups.iloc[i]['gtfs_end_date'])]
    df1 = merge_segments(df1, df2, gtfs_groups.iloc[i]['gtfs_end_date'])

df1['gtfs_start_date'] = df1['gtfs_start_date'].apply(lambda x: x.isoformat())
df1['gtfs_end_date'] = df1['gtfs_end_date'].apply(lambda x: x.isoformat())
DF = gpd.GeoDataFrame(df1, geometry='geometry')

DF.head(1)

# Add Elevation

In [ ]:
def request_elevation(lat,
                      lon,
                      units='Meters',
                      max_tries=10,
                      sec_btw_tries=1):
    usgs_url = r'https://nationalmap.gov/epqs/pqs.php?'
    usgs_params = {'output': 'json', 'x': lon, 'y': lat, 'units': units}
    for i in range(max_tries):
        try:
            usgs_request = requests.get(url=usgs_url,
                                        params=usgs_params)
            elevation = float(usgs_request.json()['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation'])
            break
        except Exception as e:
            print(e)
            elevation = None
            time.sleep(sec_btw_tries)
    return elevation


def linestring_elevation_list(geom):
    coords = list(geom.coords)
    elevations = []
    for p in coords:
        lon, lat = p[0], p[1]
        elevation = request_elevation(lat, lon)
        if elevation is not None:
            elevations.append(elevation)
    return elevations


def apply_elevation(geom, segment_id):
    elevations = linestring_elevation_list(geom)
    return {'segment_id': segment_id, 'elevation_list': elevations}

In [ ]:
unique_segment_ids = list(DF['segment_id'].unique())
print(f"There are {len(unique_segment_ids)} unique segments to get elevation for")
results = []

with concurrent.futures.ThreadPoolExecutor(max_workers=15) as executor:
    future_list = []
    for segment_id in unique_segment_ids:
        temp = DF[DF['segment_id']==segment_id].sort_values(by=['distance_m'], ascending=False).iloc[0]
        future_list.append(executor.submit(apply_elevation, temp['geometry'], segment_id))
    for future in concurrent.futures.as_completed(future_list):
        result = future.result()
        results.append(result)
        if (len(results) % 1000) == 0:
            print(len(results))

df_el = pd.DataFrame.from_records(results)
df_el['maximum_elevation'] = df_el['elevation_list'].apply(lambda x: max(x))
df_el['minimum_elevation'] = df_el['elevation_list'].apply(lambda x: min(x))
df_el['start_elevation'] = df_el['elevation_list'].apply(lambda x: x[0])
df_el['end_elevation'] = df_el['elevation_list'].apply(lambda x: x[-1])
df_el['elevation_diff'] = df_el.apply(lambda row: row['maximum_elevation'] - row['minimum_elevation'], axis=1)
df_el['elevation_change'] = df_el.apply(lambda row: row['start_elevation'] - row['end_elevation'], axis=1)
df_el.head(2)

In [ ]:
# join DF and df_el

df = pd.merge(left=DF, right=df_el, how='left', on='segment_id', validate='many_to_one')
DF = df.copy(deep=True)
DF.to_pickle(TEMP_PATH)

# INRIX XD Segments - start here

In [ ]:
DF = pd.read_pickle(TEMP_PATH)
DF.head(1)

In [ ]:
def trajectory(point1, point2):
    if isinstance(point1, Point):
        traj = math.atan2(point2.y-point1.y, point2.x-point1.x)
    else:
        traj = math.atan2(point2[1]-point1[1], point2[0]-point1[0])
    return math.degrees(traj)


def traj_dif(traj1, traj2):
    a = traj1 - traj2
    if a > 180:
        a -= 360
    elif a < -180:
        a += 360
    return abs(a)


def interpolate_linestring(segment, dist=INTERPOLATE_DIST):
    """
    segment: LineString
    dist: integer, meters
    returns: LineString
    """
    try:
        distance = 0
        add_distance = meter_to_degree(dist)
        result = []
        while distance < segment.length:
            result.append(segment.interpolate(distance))
            distance += add_distance
        return LineString(result)
    except:
        return segment
    
    
def meter_to_degree(buf):
    """
    buf: integer, represents buffer in meters
    returns: degrees
    """
    return buf / 100000


def set_tmc_buffer(df_tmc, buf=BUF):
    """
    df_tmc: geopandas dataframe of tmc shape data
    buf: integer, represents buffer in meters
    returns: df_tmc geopandas dataframe with geometry_buf set to buffered objects
    """
    df_tmc['geometry_buf'] = df_tmc['geometry'].buffer(meter_to_degree(buf))
    df_tmc.set_geometry('geometry_buf', inplace=True)
    return df_tmc


def filter_tmcs(df_tmcs_mapped, point1, point2, max_traj_diff=MAX_TRAJ_DIFF):
    result = []
    if len(df_tmcs_mapped) > 0:
        seg_traj = trajectory(point1, point2)
        for k, v in df_tmcs_mapped.iterrows():
            point1_proj = v['geometry'].project(point1)
            point2_proj = v['geometry'].project(point2)
            if point2_proj > point1_proj:
                tmc_traj = trajectory(v['geometry'].interpolate(point1_proj), v['geometry'].interpolate(point2_proj))
                if traj_dif(seg_traj, tmc_traj) < max_traj_diff:
                    result.append(v['XDSegID'])
    return result


def map_segment(df_tmc, segment, min_num=MIN_NUM):
    tmc_id_list = []
    full_segment = interpolate_linestring(segment)
    full_segment_coords = list(full_segment.coords)
    for i in range(len(full_segment_coords)-1):
        point1, point2 = Point(full_segment_coords[i]), Point(full_segment_coords[i+1])
        seg = LineString([point1, point2])
        df_tmcs_mapped = df_tmc.loc[df_tmc['geometry_buf'].contains(seg)]
        #df_tmcs_mapped = bounding_tmc_ids(df_tmc, seg)
        tmc_ids = filter_tmcs(df_tmcs_mapped, point1, point2)
        tmc_id_list += tmc_ids
    result = set()
    for tmc_id in tmc_id_list:
        if tmc_id_list.count(tmc_id) >= min_num:
            result.add(tmc_id)
    return list(result)


def add_tmcs_to_gtfs_shapes(df_inrix, df_segments):
    df_tmc = deepcopy(df_inrix)
    #df_tmc = set_tmc_buffer(df_inrix)
    result = {'segment_id': [], 'XDSegID': [], 'segment_geometry': []}
    unique_segment_ids = df_segments['segment_id'].unique()
    print(f"There are {len(unique_segment_ids)} unique segments to map")
    counter = 0
    start_time = time.time()
    for segment_id in unique_segment_ids:
        v = df_segments[df_segments['segment_id']==segment_id].sort_values(by=['distance_m'], ascending=False).iloc[0]
        tmc_id = map_segment(df_tmc, v['geometry'])
        result['segment_id'].append(segment_id)
        result['segment_geometry'].append(v['geometry'])
        if len(tmc_id) > 0:
            result['XDSegID'].append(tmc_id)
        else:
            result['XDSegID'].append(None)
        if (counter % 100) == 0:
            end_time = time.time() - start_time
            print(f"Done with {counter} segments. Took {end_time} seconds")
            start_time = time.time()
        counter += 1
    return result

In [ ]:
# load segments

df_segments = DF.copy(deep=True)

# load inrix shapes

with open(INRIX_SHAPES_PATH) as f:
    df_inrix = gpd.read_file(f)
    df_inrix = df_inrix.set_geometry('geometry')
    
print(f"number of shapes in all of tennessee: {len(df_inrix)}")
bounds = df_segments['geometry'].total_bounds
bounding_square = Polygon([(bounds[0], bounds[1]), (bounds[0], bounds[3]), (bounds[2], bounds[3]), (bounds[2], bounds[1])])
df_inrix = df_inrix[df_inrix['geometry'].intersects(bounding_square)]
print(f"number of shapes in chattanooga region: {len(df_inrix)}")
#df_inrix.head(2)

# process

df_inrix = set_tmc_buffer(df_inrix, buf=BUF)
result = add_tmcs_to_gtfs_shapes(df_inrix, df_segments)

df_result = pd.DataFrame.from_records(result)


In [ ]:
df_result = df_result[['XDSegID', 'segment_id']]
df = pd.merge(left=DF, right=df_result, how='left', on='segment_id', validate='many_to_one')
DF = df.copy(deep=True)
DF['XDSegID'] = DF['XDSegID'].apply(lambda x: format_xdseg(x))
DF.to_pickle(TEMP_PATH)

# OSM

In [ ]:
def format_seg_col(x):
    if isinstance(x, str):
        x = x.replace("[", "")
        x = x.replace("]", "")
        x = x.replace("'", "")
        return [int(y) for y in x.split(",")]
    else:
        return x
    
def format_col(x):
    if isinstance(x, str):
        return [int(y) for y in x.split(";")]
    else:
        return x
    
def get_osm_ways(x, df_xd_osm_map):
    result = []
    if isinstance(x, list):
        for xd in x:
            way = df_xd_osm_map[df_xd_osm_map['XDSegID']==xd]
            if len(way) == 1:
                result.append(way.iloc[0]['OSMWayIDs'])
            else:
                continue
    else:
        return x
    if len(result) > 0:
        return list(set([item for sublist in result for item in sublist]))
    else:
        return None
    
def get_osm_fclasses(x, df_osm):
    result = []
    if isinstance(x, list):
        for osm_way in x:
            fclass = df_osm[df_osm['osm_id']==osm_way]
            if len(fclass) == 1:
                result.append(fclass.iloc[0]['fclass'])
    else:
        return x
    return list(set(result))

def format_xdseg(x):
    try:
        return [int(y) for y in x]
    except:
        return x

In [ ]:
df_xd = df_result.copy(deep=True)
df_xd['XDSegID'] = df_xd['XDSegID'].apply(lambda x: format_xdseg(x))

df_osm = gpd.read_file(os.path.join(os.getcwd(), 'data', 'inrix', 'osm', 'chattanooga_osm.shp'))
df_osm['osm_id'] = df_osm['osm_id'].astype(int)

df_xd_osm_map = pd.read_csv(os.path.join(os.getcwd(), 'data', 'inrix', 'xd_to_osm.csv'))[['XDSegID', 'OSMWayIDs']]
df_xd_osm_map['OSMWayIDs'] = df_xd_osm_map['OSMWayIDs'].apply(lambda x: format_col(x))
df_xd_osm_map['XDSegID'] = df_xd_osm_map['XDSegID'].astype(int)

df_xd['osm_ways'] = df_xd['XDSegID'].apply(lambda x: get_osm_ways(x, df_xd_osm_map))

df_xd['osm_way_fclasses'] = df_xd['osm_ways'].apply(lambda x: get_osm_fclasses(x, df_osm))

DF = DF.drop(columns=['XDSegID'])
DF = pd.merge(left=DF, right=df_xd, how='left', on='segment_id', validate='many_to_one')
DF.to_pickle(TEMP_PATH)

# HERE TMC Mapping

In [ ]:
def query_here_shapes(client, db_name='data-class', collection_name='here.chattanooga.shapes'):
    df_tmc = gpd.GeoDataFrame(list(client[db_name][collection_name].find()))
    df_tmc['geometry'] = df_tmc['geom'].apply(lambda x: flatten_geoms(x))
    df_tmc.drop(columns=['_id'], inplace=True)
    return df_tmc


def get_mongo_connection():
    with open(CONFIG_PATH) as file:
        config = json.load(file)

    mongo_url = "mongodb://{}:{}@{}:{}/?authSource={}".format(config["user"],
                                                              config["password"],
                                                              config["host"],
                                                              config["port"],
                                                              config["authenticationDatabase"])

    client = pymongo.MongoClient(mongo_url)
    return client


def flatten_geoms(x):
    geom = x['coordinates']
    if x['type'] == "MultiLineString":
        result = []
        for linestring in geom:
            result += linestring
    else:
        result = geom
    return LineString(result)

In [ ]:
# get tmc shapes

CONFIG_PATH = os.path.join(os.getcwd(), "config", "config.json")
client = get_mongo_connection()
df_tmc = query_here_shapes(client)
client.close()

# load segments
df_segments = DF.copy(deep=True)


# process
df_tmc = set_tmc_buffer(df_tmc, buf=BUF)
df_tmc['XDSegID'] = df_tmc['tmc_id']
result = add_tmcs_to_gtfs_shapes(df_tmc, df_segments)

df = pd.DataFrame.from_records(result)
df['tmc_id'] = df['XDSegID']
df = df.drop(columns=['XDSegID', 'segment_geometry'])


In [ ]:
DF = pd.merge(left=DF, right=df, how='left', on='segment_id', validate='many_to_one')
DF.to_pickle(SEGMENTS_PATH)
DF.head(1)

In [ ]:
print(f"length of segments: {len(DF)}, null osm ways: {DF['osm_way_fclasses'].isnull().sum()}, null XDSegIDs: {DF['XDSegID'].isnull().sum()}, null tmc ids: {DF['tmc_id'].isnull().sum()}")

In [ ]:
print(f"length of segments: {len(DF)}, null osm ways: {DF['osm_way_fclasses'].isnull().sum()}, null XDSegIDs: {DF['XDSegID'].isnull().sum()}, null tmc ids: {DF['tmc_id'].isnull().sum()}")